In [1]:
import sys
import os
import pandas as pd

sys.path.append('..')

In [4]:
from icenode.train_snonet_diag import SNONETDiag
from icenode.train_gram import GRAM
from icenode.train_retain import RETAIN

%load_ext autoreload
%autoreload 2

/home/asem/.local/lib/python3.8/site-packages/jax/experimental/optimizers.py:28: FutureWarning: jax.experimental.optimizers is deprecated, import jax.example_libraries.optimizers instead
  warnings.warn('jax.experimental.optimizers is deprecated, '


## Define Directories

In [5]:
HOME = os.environ.get('HOME')

# MIMIC-III Dataset Directory
mimic3_dir = f'{HOME}/GP/ehr-data/mimic3-transforms'

# ICE-NODE/M trained on MIMIC-III training partition (70%)
icenode_m3_dir = f'{HOME}/GP/ehr-data/icenode-m3-exp/v0.1.23M3_snonet_diag_M/frozen_trial_3376'

# RETAIN trained on MIMIC-III training partition (70%)
retain_m3_dir = f'{HOME}/GP/ehr-data/icenode-m3-exp/v0.1.23M3_retain_M/frozen_trial_142'

# GRU=GRAM/M trained on MIMIC-III training partition (70%)
gru_m3_dir = f'{HOME}/GP/ehr-data/icenode-m3-exp/v0.1.23M3_gram_M/frozen_trial_615'

# GRAM=GRAM/G trained on MIMIC-III training partition (70%)
gram_m3_dir = f'{HOME}/GP/ehr-data/icenode-m3-exp/v0.1.23M3_gram_G/frozen_trial_442'

## Patient Interface for each Model

In [6]:
icnode_patient_interface = SNONETDiag.create_patient_interface(mimic3_dir, 'M3')
retain_patient_interface = RETAIN.create_patient_interface(mimic3_dir, 'M3')
# Works for both GRAM/M and GRAM/G
gram_patient_interface = GRAM.create_patient_interface(mimic3_dir, 'M3')


2022-01-28 21:47:20.271935: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


## Dataset Partitioning

In [7]:
import random

# seed 42 is used in all our experiments in this work.
rng = random.Random(42)
subjects_id = list(gram_patient_interface.subjects.keys())
rng.shuffle(subjects_id)

# splits = train:val:test = 0.7:.15:.15
splits = int(.7 * len(subjects_id)), int(.85 * len(subjects_id))

train_ids = subjects_id[:splits[0]]
valid_ids = subjects_id[splits[0]:splits[1]]
test_ids = subjects_id[splits[1]:]

## Load Configs and Trained Params

In [8]:
from icenode.utils import load_config, load_params

icenode_config = load_config(f'{icenode_m3_dir}/config.json')
icenode_params = load_params(f'{icenode_m3_dir}/step0100_params.pickle')

retain_config = load_config(f'{retain_m3_dir}/config.json')
retain_params = load_params(f'{retain_m3_dir}/step0100_params.pickle')

gru_config = load_config(f'{gru_m3_dir}/config.json')
gru_params = load_params(f'{gru_m3_dir}/step0100_params.pickle')

gram_config = load_config(f'{gram_m3_dir}/config.json')
gram_params = load_params(f'{gram_m3_dir}/step0100_params.pickle')

## Create Model Objects

In [9]:
icenode_config

{'emb': {'diag': {'embeddings_dim': 250}, 'kind': 'matrix'},
 'model': {'init_depth': 1,
  'max_odeint_days': 3600,
  'ode_depth': 0,
  'ode_dyn': 'gru',
  'ode_init_var': 9.774585690261578e-05,
  'ode_timescale': 859.5151988814158,
  'ode_with_bias': False,
  'state_size': 150,
  'tay_reg': 4},
 'training': {'batch_size': 2,
  'diag_loss': 'softmax',
  'epochs': 10,
  'loss_mixing': {'L_diag': 0.5260797315618758,
   'L_dyn': 0.8058584767919931,
   'L_l1': 1.5313215497766123e-07,
   'L_l2': 2.164606010146564e-06},
  'lr': 0.00038483993711989643,
  'optimizer': 'adam'}}

In [10]:
icenode = SNONETDiag.create_model(icenode_config, icnode_patient_interface, train_ids, None)
retain = RETAIN.create_model(retain_config, retain_patient_interface, train_ids, None)
gru = GRAM.create_model(gru_config, gram_patient_interface, train_ids, None)
gram = GRAM.create_model(gram_config, gram_patient_interface, train_ids, None)

In [11]:
code_partitions = GRAM.code_partitions(gram_patient_interface, train_ids) 

In [12]:
from icenode.metrics import evaluation_table
res = icenode.eval(icenode_config['training']['loss_mixing'], icenode_params, test_ids)

In [13]:
eval_df = evaluation_table({'TST': res}, code_partitions)

/home/asem/.local/lib/python3.8/site-packages/jax/_src/numpy/lax_numpy.py:5336: UserWarning: 'kind' argument to argsort is ignored; only 'stable' sorts are supported.
  warnings.warn("'kind' argument to argsort is ignored; only 'stable' sorts "


In [14]:
eval_df

(                             TST
 MACRO-AUC               0.977391
 MICRO-AUC               0.977010
 accuracy                0.968672
 all_points_count      555.000000
 diag_loss              46.518295
 dyn_loss              192.595169
 dyn_loss_per_week       0.003134
 f1-score                0.461468
 fn                      0.002776
 fp                      0.028551
 l1_loss             53746.003906
 l2_loss              6450.216797
 loss                   46.543015
 nfe_per_week            1.777862
 nfex1000              109.241997
 npv                     0.997102
 postjump_diag_loss     45.867481
 pre_ACC-P0              0.417352
 pre_ACC-P1              0.636641
 pre_ACC-P2              0.738486
 pre_ACC-P3              0.883964
 pre_ACC-P4              0.960646
 precision               0.319779
 predictable_count    9351.000000
 prejump_diag_loss      47.240746
 recall                  0.828614
 specificity             0.970979
 tn                      0.955250
 tp           

## Analyse AUC for Each Admission in the Test Partition

In [15]:
icenode_auc_df = icenode.admissions_auc_scores(icenode_params, test_ids)

In [16]:
retain_auc_df = retain.admissions_auc_scores(retain_params, test_ids)

In [17]:
gru_auc_df = gru.admissions_auc_scores(gru_params, test_ids)

In [18]:
gram_auc_df = gram.admissions_auc_scores(gram_params, test_ids)

In [29]:
gram_auc_df.columns

Index(['SUBJECT_ID', 'HADM_ID', 'HADM_IDX', 'AUC', 'N_CODES'], dtype='object')

In [30]:
icenode_auc_df.columns = ['SUBJECT_ID', 'HADM_ID', 'HADM_IDX', 'ICENODE_AUC', 'N_CODES', 'DAYS_AHEAD', 'INTERVAL', 'ICENODE_NFE']

retain_auc_df.columns = ['SUBJECT_ID', 'HADM_ID', 'HADM_IDX', 'RETAIN_AUC', 'N_CODES']

gru_auc_df.columns = ['SUBJECT_ID', 'HADM_ID', 'HADM_IDX', 'GRU_AUC', 'N_CODES']

gram_auc_df.columns = ['SUBJECT_ID', 'HADM_ID', 'HADM_IDX', 'GRAM_AUC', 'N_CODES']


In [32]:
icenode_auc_df.HADM_ID.nunique()

1605

In [33]:
icenode_auc_df

,SUBJECT_ID,HADM_ID,HADM_IDX,ICENODE_AUC,N_CODES,DAYS_AHEAD,INTERVAL,ICENODE_NFE
0,24198,135189,1,0.976323,9.0,1,1,8.0
1,24198,135189,2,0.976787,9.0,2,1,8.0
2,24198,135189,3,0.977097,9.0,3,1,8.0
3,24198,135189,4,0.976787,9.0,4,1,8.0
4,24198,135189,5,0.976942,9.0,5,1,8.0
...,...,...,...,...,...,...,...,...
9346,11234,150220,2,0.964360,8.0,1267,1266,98.0
9347,11234,150220,3,0.996697,8.0,1268,1,8.0
9348,11234,150220,4,0.998261,8.0,1269,1,8.0
9349,11234,150220,5,0.998435,8.0,1270,1,8.0


In [35]:
icenode_auc_df2 = icenode_auc_df.drop_duplicates(['SUBJECT_ID', 'HADM_ID'])

In [37]:
icenode_auc_df2.ICENODE_AUC.mean()

0.9572364170139803

In [38]:
icenode_auc_df2

,SUBJECT_ID,HADM_ID,HADM_IDX,ICENODE_AUC,N_CODES,DAYS_AHEAD,INTERVAL,ICENODE_NFE
0,24198,135189,1,0.976323,9.0,1,1,8.0
13,24198,157166,14,0.971526,9.0,217,204,32.0
20,24198,116391,21,0.978373,7.0,336,113,26.0
25,97441,138440,1,0.977877,21.0,1,1,8.0
32,97441,194679,8,0.943302,16.0,39,32,20.0
...,...,...,...,...,...,...,...,...
9326,59496,158272,10,0.964569,12.0,20,11,14.0
9333,1949,122008,1,0.976765,11.0,1,1,8.0
9337,1949,196537,5,0.863499,13.0,10,6,14.0
9345,11234,133876,1,0.997925,4.0,1,1,8.0
